In [13]:
import re
import requests
import pandas as pd
from datetime import datetime, timedelta, date

In [14]:
start_of_season = date(2020,8,31)
days = abs(start_of_season-date.today()).days
week = days//7

In [15]:
nflweek = str(week) + 'W-HCAP'
url = 'https://www.bovada.lv/services/sports/event/v2/events/A/description/football/nfl'
r = requests.get(url)
j = r.json()  


In [16]:
results_list = j[0]



In [17]:
## defines how many games are being played this week
## we do this by grabbing the output from the url listed above which is a json
## and then we ask it to return specific data from an element called 'events'

games_list = results_list['events']


## we then loop through the data in the 'events' node and find any data where 
## the 'key' within the 'markets' node within the 'displaygroup' node has a value of '2W-HCAP'
## that means it's a "2 way handicap" or a spread. If we didn't have this our spreads below
## would return a ton of different stuff like over/unders, prop bets, etc.

num_of_games = 0
for i in range(len(games_list)):
    try:
        if (games_list[i]['displayGroups'][0]['markets'][1]['key'] == '2W-HCAP'): # and games_list[i]['notes'] == ''):
            num_of_games = num_of_games + 1
    except:
        continue
               
        
#num_of_games = 16


In [18]:
## loops through the json to pull out the spreads

defaults = {}
defaultfinal = []

for i in range(int(num_of_games)):
    try:
        nfl_game = games_list[i]
        team_a = nfl_game['displayGroups'][0]['markets'][1]['outcomes'][0]['description']
        team_b = nfl_game['displayGroups'][0]['markets'][1]['outcomes'][1]['description']
        team_a_loc = nfl_game['displayGroups'][0]['markets'][1]['outcomes'][0]['type']
        team_a_spread = nfl_game['displayGroups'][0]['markets'][1]['outcomes'][0]['price']['handicap']
        team_b_spread = nfl_game['displayGroups'][0]['markets'][1]['outcomes'][1]['price']['handicap']
        defaults.update (homeTeam = team_b, homeSpread = team_b_spread, awayTeam = team_a, awaySpread = team_a_spread)
        defaultfinal.append(dict(defaults))
    except:
        print('something dumb happened')
    
    

In [19]:
## converts the dictionary to a dataframe
def spreads():
    df = pd.DataFrame(defaultfinal)
    return df
    

In [20]:
spreads()

,homeTeam,homeSpread,awayTeam,awaySpread
0,Kansas City Chiefs,-11.5,New England Patriots,11.5
1,Green Bay Packers,-6.0,Atlanta Falcons,6.0
2,Chicago Bears,6.0,Tampa Bay Buccaneers,-6.0
3,New York Jets,7.0,Arizona Cardinals,-7.0
4,Baltimore Ravens,-13.5,Cincinnati Bengals,13.5
5,Houston Texans,-6.5,Jacksonville Jaguars,6.5
6,Washington Football Team,9.0,Los Angeles Rams,-9.0
7,Pittsburgh Steelers,-7.5,Philadelphia Eagles,7.5
8,Cleveland Browns,3.0,Indianapolis Colts,-3.0
9,Dallas Cowboys,-9.5,New York Giants,9.5
